In [1]:
# Last run at d94de94

In [ ]:
# Verify metabolites belonging to the same BGC are in general more similar than metabolites from different BGCs

In [2]:
import csv
import os
import numpy

In [3]:
mibig_structures = "/home/grimur/_iokr/data/mibig/compunds_structures_1.4.csv"

bgc_structures = {}

with open(mibig_structures) as f:
    r = csv.reader(f)
    header = next(r)
    for line in r:
        bgc_id, bgc_name, smiles, pubchem = line
    
        if smiles.strip() == "":
            continue

        if bgc_id in bgc_structures:
            bgc_structures[bgc_id].append(smiles)
        else:
            bgc_structures[bgc_id] = [smiles]        


In [4]:
import sys
sys.path.append('/home/grimur/git/nplinker/nplinker/prototype')
from nplinker.scoring.iokr import mk_fprints

['/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/cdk_pywrapper']
Server process already running: True


In [12]:
smiles_fingerprints = {}
for bgc, smiles_list in bgc_structures.items():
    for smiles in smiles_list:
        if smiles not in smiles_fingerprints:
            fp = mk_fprints.fingerprint_from_smiles(smiles)
            smiles_fingerprints[smiles] = fp


In [6]:
import numpy

def fp_vector_kernel(v1, v2):
    v1 = v1.astype('bool')
    v2 = v2.astype('bool')
    tanimoto_or = numpy.count_nonzero(numpy.logical_or(v1, v2))
    tanimoto_and = numpy.count_nonzero(numpy.logical_and(v1, v2))
    if tanimoto_and is 0:
        tanimoto = 0
    else: 
        tanimoto = tanimoto_or / tanimoto_and
    return gaussian(tanimoto ** 2)

def l2_gaussian(v1, v2, gamma=0.01):
    d_sq = numpy.sum(numpy.power(v1 - v2, 2))
    return gaussian(d_sq, gamma)

def gaussian(x, gamma=0.01):
    return numpy.exp(-(gamma * x))

def pfam_vector_kernel(v1, v2):
    return l2_gaussian(numpy.array(v1), numpy.array(v2))



In [13]:
# The SMILES equivalence classes are the union of all BGC-based SMILES equivalence classes.
# i.e. if A, B are SMILES for BGC b1, and A, C are SMILES for BGC b2, 
# then A, B, C all belong to the same equivalence class.

smiles_equiv_classes = {}
total_smiles_list = set([])

for bgc, smiles_list in bgc_structures.items():
    total_smiles_list = total_smiles_list.union(smiles_list)
    for smiles in smiles_list:
        if smiles not in smiles_equiv_classes:
            smiles_equiv_classes[smiles] = set([])
        smiles_equiv_classes[smiles] = smiles_equiv_classes[smiles].union(smiles_list)
        
total_smiles_list = list(total_smiles_list)


In [21]:

equivalent_dists = []
unique_dists = []

for r in range(100000):
    smiles_1 = numpy.random.choice(total_smiles_list)
    smiles_2 = numpy.random.choice(total_smiles_list)
    fv_1 = smiles_fingerprints[smiles_1]
    fv_2 = smiles_fingerprints[smiles_2]

    dist = fp_vector_kernel(fv_1, fv_2)
    if smiles_1 in smiles_equiv_classes[smiles_2]:
        equivalent_dists.append(dist)
    else:
        unique_dists.append(dist)

In [26]:
print('equiv: ', numpy.mean(equivalent_dists))
print('unique: ', numpy.mean(unique_dists))

equiv:  0.9731761841730556
unique:  0.7450948074396773


In [32]:
import scipy.stats
print(scipy.stats.ttest_ind(equivalent_dists, unique_dists))

Ttest_indResult(statistic=17.007231947203504, pvalue=8.956731615948826e-65)


In [36]:
print('Total number of BGCs:', len(bgc_structures.keys()))
print('Total number of metabolites:', len(total_smiles_list))

Total number of BGCs: 1205
Total number of metabolites: 1452


In [37]:
count = 0
for bgc, smiles in bgc_structures.items():
    for s in smiles:
        count += 1
print('Total number of pairs:', count)

Total number of pairs: 1692
